In [14]:
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'Is MPS available? {torch.backends.mps.is_available()}')
print(f'Does MPS exists? {torch.backends.mps.is_built()}')

PyTorch version: 2.6.0
Is MPS available? True
Does MPS exists? True


# Introducción

*Los **tensores** son una de las estructuras de datos más importantes dentro de PyTorch. Son arrays multidimensionales, similares a un `ndarray` de NumPy, pero con capacidades adicionales que los hacen muy útiles para el entrenamiento de modelos de Deep Learning.*

## Crear tensores

### Escalares

In [2]:
# Create a scalar
scalar = torch.tensor(7.0)
scalar

tensor(7.)

*Si quisieramos extraer el número entero de nuestro tensor tendríamos que utilizar el método `item()`.*

In [3]:
# Get a tensor back as an integer or float
print(f'Escalar: {scalar.item()}. Type: {type(scalar.item())}')

Escalar: 7.0. Type: <class 'float'>


### Vectores

*Los vectores son arrays unidimensionales. Utilizando el atributo `ndim` podemos validar que el tensor que creamos tiene una única dimensión. Podemos mirar con mas detalle las dimensiones del tensor con el atributo `shape` (notar que su dimensión tiene tamaño dos).*

In [4]:
# Create a vector
vector = torch.tensor([7, 7])
vector

tensor([7, 7])

In [5]:
# Dimensions
print(f'Number of dimensions: {vector.ndim}')

Number of dimensions: 1


In [6]:
print(f'Shape: {vector.shape}')

Shape: torch.Size([2])


### Matrices

*Las matrices son arrays **bi-dimensionales**, en donde la primera dimensión representa las filas y la segunda dimensión representa las columnas. El tamaño que tienen esas dimensiones hacen referencia la cantidad de filas y la cantidad de columnas de la matriz.*

In [7]:
MATRIX = torch.tensor([[7, 8], [8, 9]])
MATRIX

tensor([[7, 8],
        [8, 9]])

In [8]:
print(f'Number of dimensions: {MATRIX.ndim}')

Number of dimensions: 2


In [9]:
print(f'Shape: {MATRIX.shape}')

Shape: torch.Size([2, 2])


### Tensores

*Si bien los vectores y las matrices que vimos anteriormente también pueden ser llamados "tensores" (porque esta es la principal estructura de datos en PyTorch), los tensores propiamente dichos son arrays **n-dimensionales**. Por ejemplo, en el código debajo tenemos un tensor de tres dimensiones, aunque se puede extender fácilmente a más dimensiones.*

In [10]:
# Create a tensor
TENSOR = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
TENSOR

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [11]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [12]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([1, 3, 3])


*Para crear un tensor a partir de otro(s) tensor(es) como input(s), es necesario utilizar funciones como `stack()` o `vstack()`. Esto es así porque la clase `torch.tensor()` solo acepta objetos nativos de Python.*

*En este caso utilizamos la función `stack()` que nos permite concatenar tensores sobre una nueva dimensión. Si el argumento `dim = 0` entonces crea una primera nueva dimensión para concatenar los tensores (en nuestro caso, como tenemos tensores de dos dimensiones, crea una tercera dimensión).*

In [13]:
MATRIX_1 = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
MATRIX_2 = torch.tensor([[10, 11, 12], [13, 14, 15], [16, 17, 18]])

# The torch.tensor() class expects Python native data types, so in order to create a tensor from two tensors
TENSOR = torch.stack([MATRIX_1, MATRIX_2])
TENSOR

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]]])

In [14]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [15]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([2, 3, 3])


### Tensores aleatorios

*Por qué queremos aprender a generar tensores de forma aleatoria? Porque los pesos de las redes neuronales se inicializan de manera aleatoria. Entonces, al comenzar el entrenamiento de nuestra red neuronal debemos pasarle un tensor que represente los pesos iniciales, el cual tenemos que poder generar de manera aleatoria.*

In [16]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(2, 3))
random_tensor

tensor([[0.9820, 0.1792, 0.5302],
        [0.3143, 0.9677, 0.3722]])

In [17]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 2
Tamaños: torch.Size([2, 3])


In [18]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(10, 2, 3))
random_tensor

tensor([[[0.2588, 0.2084, 0.9781],
         [0.3258, 0.2819, 0.5629]],

        [[0.4077, 0.2434, 0.9305],
         [0.1911, 0.3808, 0.4754]],

        [[0.4987, 0.9741, 0.5369],
         [0.6891, 0.7910, 0.6819]],

        [[0.1176, 0.5191, 0.0440],
         [0.1203, 0.3199, 0.0255]],

        [[0.9614, 0.4660, 0.3628],
         [0.3325, 0.5683, 0.8378]],

        [[0.5982, 0.1143, 0.4015],
         [0.9410, 0.9701, 0.0583]],

        [[0.7335, 0.1181, 0.8162],
         [0.0412, 0.9402, 0.4521]],

        [[0.1831, 0.6347, 0.4051],
         [0.8247, 0.2526, 0.4277]],

        [[0.0990, 0.1635, 0.3707],
         [0.6705, 0.2528, 0.4427]],

        [[0.5400, 0.1594, 0.6117],
         [0.0854, 0.3322, 0.8924]]])

In [19]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 3
Tamaños: torch.Size([10, 2, 3])


*Las imagenes se pueden representar como tensores de tres dimensiones, en donde la primer dimensión representa los canales de colores (RGB) y la segunda y tercer dimensión representa la cantidad de píxeles de la imágen. Por ejemplo, si tenemos una imagen a color de 1024x1024 y la convertimos en un tensor, entonces tendríamos un tensor de tres dimensiones con tamaño `torch.Size([3, 1024, 1024])`.*

In [20]:
random_image_tensor = torch.rand(size=(3, 1024, 1024))
random_image_tensor

tensor([[[0.8925, 0.6278, 0.2613,  ..., 0.4122, 0.0031, 0.0109],
         [0.7298, 0.3350, 0.7771,  ..., 0.2389, 0.4367, 0.8209],
         [0.6178, 0.8236, 0.1508,  ..., 0.2359, 0.1650, 0.5251],
         ...,
         [0.9389, 0.7546, 0.8447,  ..., 0.7073, 0.7264, 0.6078],
         [0.9003, 0.3145, 0.7920,  ..., 0.9895, 0.6250, 0.9059],
         [0.0570, 0.0024, 0.7315,  ..., 0.8897, 0.2080, 0.4780]],

        [[0.3806, 0.2466, 0.8097,  ..., 0.5088, 0.3978, 0.2602],
         [0.2367, 0.1366, 0.5563,  ..., 0.9141, 0.0696, 0.8193],
         [0.2554, 0.8213, 0.7212,  ..., 0.8288, 0.9923, 0.9311],
         ...,
         [0.1223, 0.1816, 0.2465,  ..., 0.1422, 0.1325, 0.9611],
         [0.1939, 0.0018, 0.7103,  ..., 0.4424, 0.0356, 0.4601],
         [0.2124, 0.8028, 0.4986,  ..., 0.3108, 0.6996, 0.8184]],

        [[0.4243, 0.6175, 0.6683,  ..., 0.4923, 0.3915, 0.5057],
         [0.5467, 0.3246, 0.8679,  ..., 0.5577, 0.5031, 0.5486],
         [0.2825, 0.4570, 0.7711,  ..., 0.0871, 0.1571, 0.

In [21]:
print(f'Dimensiones: {random_image_tensor.ndim}')
print(f'Shape: {random_image_tensor.shape}')

Dimensiones: 3
Shape: torch.Size([3, 1024, 1024])


### Unos y ceros

*En algunas ocaciones nos gustaría tener tensores que solo tienen ceros o unos. Los podemos crear con las funciones `torch.zeros()` o `torch.ones()`.*

In [22]:
zeros = torch.zeros(size=(2, 3, 4))
zeros

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [23]:
ones = torch.ones(size=(2, 3, 4))
ones

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

### Tensor-like objects

In [24]:
range_tensor = torch.arange(start=0, end=1000, step=36)
range_tensor

tensor([  0,  36,  72, 108, 144, 180, 216, 252, 288, 324, 360, 396, 432, 468,
        504, 540, 576, 612, 648, 684, 720, 756, 792, 828, 864, 900, 936, 972])

In [25]:
print(f'Dimension: {range_tensor.ndim}')
print(f'Shape: {range_tensor.shape}')
print(f'Dtype: {range_tensor.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


*Cuando decimos que vamos a crear o que creamos un objeto que es "tensor-like", nos referimos a crear un nuevo tensor que tenga la misma dimensión, tamaño y tipo de dato que otro tensor ya existente, pero con otros valores.*

*Por ejemplo, podemos creamos un nuevo tensor a partir del tensor `range_tensor` que tenga la misma dimensión, tamaño y tipo de dato, pero que la información que contiene sea distinta (solo tenga ceros).*

In [26]:
zeros = torch.zeros_like(input=range_tensor)
zeros

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [27]:
print(f'Dimension: {zeros.ndim}')
print(f'Shape: {zeros.shape}')
print(f'Dtype: {zeros.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


### Customized tensors

*Por default, PyTorch crea los tensores utilizando el tipo de dato `torch.float32`. Sin embargo, hay muchos más tipos de datos [disponibles](https://pytorch.org/docs/stable/tensors.html#data-types), los cuales se diferencian principalmente por la precisión númerica que nos aportan. Por ejemplo, el tipo de dato `torch.float64` tiene mucho más precisión numérica que el tipo de dato `torch.float32` (casi el doble de decimales), pero consume mucho más memoria.*

*Podemos cambiar el tipo de dato de nuestro tensor utilizando el argumento `dtype`.*

In [3]:
TENSOR_0 = torch.tensor([3., 2., 1.], dtype=torch.float64)
TENSOR_0

tensor([3., 2., 1.], dtype=torch.float64)

In [4]:
TENSOR_0.dtype

torch.float64

In [5]:
# Change tensor's datatype
TENSOR_1 = TENSOR_0.type(torch.float16) # or use torch.Tensor.to()
TENSOR_1

tensor([3., 2., 1.], dtype=torch.float16)

*Cuando realizamos operaciones matemáticas entre tensores que fueron definidos con distintos tipos de datos, PyTorch utiliza una **regla de promoción** para definir el tipo de dato que tendrá el tensor resultante (por lo general, tendrá el tipo de dato con mayor precisión). Por ejemplo, si multiplicamos un tensor con tipo de dato `torch.float32` y otro con tipo de dato `torch.float64`, el tensor que obtenemos después de la operación es `torch.float64`.*

*Esto es algo importante a tener en cuenta porque:*
1. *Podemos llegar a consumir más memoria de la esperada si las operaciones que realizamos están constantemente incrementando la precisión númerica de nuestros tensores.*
2. *Las operaciones entre tensores con distintos tipos de datos suelen ser menos performantes porque PyTorch se debe encargar también de convertir los datos.*
3. *Algunos dispositivos tienen soporte limitado para algunos tipos de datos (e.g., las GPU no soportan tensores con tipos de datos `torch.float64` o las `mps` no soportan los tensores con tipos de datos `torch.float16`).*

In [7]:
TENSOR_0 * TENSOR_1

tensor([9., 4., 1.], dtype=torch.float64)

*Podemos también definir el **dispositivo** en donde se guarda y ejecuta el tensor. Hay varios tipos de dispositivos que podemos utilizar, pero los más utilizados son:*
1. *`cpu`: nos permite guardar y ejecutar los tensores en la memoria RAM de nuestra computadora o máquina virtual.*
2. *`cuda`: nos permite guardar y ejecutar los tensores en la GPU. Si tenemos disponibles más de una GPU, podemos especificar cual queremos utilizar, e.g.,  `cuda:0` (primer GPU), `cuda:1` (segunda GPU), `cuda:2` (tercera GPU), etc.*
3. *`mps`: nos prmite utilizar la GPU integrada en las computadoras con chips Apple Silicon (similar a como `cuda` nos permite utilizar las GPUs de NVIDIA).*

*Por qué es importante definir el dispositivo?:*
1. *Si tenemos dos tensores que están guardados en dos dispositivos distintos no podemos realizar operaciones entre ellos.*
2. *Si queremos guardar y ejecutar los tensores en la GPU, tenemos que seleccionar el dispositivo `cuda`.*
3. *Mover tensores de un dispositivo a otro puede ser computacionalmente costoso.*

In [32]:
TENSOR.device

device(type='cpu')

*Podemos elegir el dispositivo en donde queremos guardar y ejecutar el tensor al momento de crearlo:*

In [33]:
# Define a torch.device
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a new tensor with specified device
TENSOR_2 = torch.rand(size=(3, 1024, 1024), device='cpu')
print(f'Tensor: {TENSOR_2}')
print(f'Device: {TENSOR_2.device}')

Tensor: tensor([[[0.8056, 0.8615, 0.5046,  ..., 0.4668, 0.6164, 0.5404],
         [0.9239, 0.7019, 0.5254,  ..., 0.9522, 0.9213, 0.4076],
         [0.1358, 0.5212, 0.0534,  ..., 0.4795, 0.7317, 0.4672],
         ...,
         [0.5984, 0.7380, 0.7738,  ..., 0.5883, 0.6407, 0.8307],
         [0.0475, 0.6639, 0.4176,  ..., 0.7113, 0.8032, 0.8278],
         [0.4835, 0.5502, 0.1951,  ..., 0.0073, 0.2049, 0.3122]],

        [[0.9995, 0.7859, 0.4808,  ..., 0.6156, 0.6079, 0.5768],
         [0.0739, 0.1870, 0.6023,  ..., 0.1718, 0.8409, 0.7399],
         [0.8333, 0.6206, 0.5385,  ..., 0.7740, 0.1770, 0.2031],
         ...,
         [0.7577, 0.6314, 0.8505,  ..., 0.2738, 0.9426, 0.0358],
         [0.6079, 0.5930, 0.8151,  ..., 0.3913, 0.6865, 0.9591],
         [0.4419, 0.4249, 0.2461,  ..., 0.5989, 0.2518, 0.7804]],

        [[0.1224, 0.4576, 0.3832,  ..., 0.1964, 0.1862, 0.6033],
         [0.0582, 0.7010, 0.5833,  ..., 0.8393, 0.8065, 0.6335],
         [0.3276, 0.5404, 0.9176,  ..., 0.3553, 0.

*Podemos también utilizar la función `torch.Tensor.to` para cambiar el tipo de dato o el dispositivo de un tensor (notar que es necesario re-asignar el tensor a la variable):*

In [ ]:
TENSOR_2 = TENSOR_2.to(device='mps')
print(f'Device: {TENSOR_2.device}')

*Otro argumento muy importante que tienen los tensores en PyTorch es el argumento `requires_grad`. Cuando definimos `requires_grad=True` en un tensor, lo que le estamos pidiendo a PyTorch es que guarde todas las operaciones realizadas sobre un tensor, para así poder calcular los gradientes. Esto es muy útil cuando queremos entrenar un modelo de Deep Learning, ya que nos permite calcular los gradientes de los pesos del modelo de manera automática utilizando backpropagation.*

*Notar que en el siguiente ejemplo creamos una nueva variable, `y`, que es igual a la suma de los elementos del tensor `X`. Luego, para calcular los gradientes de `y` con respecto a los elementos en `X`, llamamos a la función `.backward()` desde la variable `y`. Sin embargo, los gradientes los podemos encontrar en el tensor `X` (eso tiene sentido porque los gradientes nos dicen como cambia `y` cuando cambia un elemento de `X`.*

In [13]:
# Creates a tensor and operates on it
X = torch.tensor([3., 2., 1.], requires_grad=True, device='mps')

# Sum all elements of the tensor and compute the gradients
y = X.sum()
y.backward()
print(f'Gradients after automatic differentiation: {TENSOR_0.grad}')

Gradients after automatic differentiation: tensor([1., 1., 1.], device='mps:0')
